In [1]:
from datetime import datetime
from read_jleague_matches import read_all_matches 

In [2]:
CATEGORY = 3

all_matches = read_all_matches(CATEGORY)
all_matches.to_csv(f'match_result-J{CATEGORY}-{datetime.now().strftime("%Y%m%d")}.csv')

all_matches

,match_date,section_no,match_index_in_section,start_time,stadium,home_team,home_goal,away_goal,away_team
0,2021-03-14,1,1,13:03,とうスタ,藤枝,0,0,福島
1,2021-03-14,1,2,13:03,ニッパツ,富山,1,0,YS横浜
2,2021-03-14,1,3,13:03,夢スタ,熊本,1,0,今治
3,2021-03-14,1,4,13:03,ユニスタ,岩手,1,0,宮崎
4,2021-03-14,1,5,14:03,長良川,八戸,0,0,岐阜
...,...,...,...,...,...,...,...,...,...
2,2021-11-28,29,3,未定,長良川,藤枝,,,岐阜
3,2021-11-28,29,4,未定,夢スタ,八戸,,,今治
4,2021-11-28,29,5,未定,白波スタ,長野,,,鹿児島
5,2021-11-28,29,6,未定,ユニスタ,熊本,,,宮崎
